In [ ]:
!pip install dash plotly pandas


In [43]:
import pandas as pd
data = {
    "Année" : [2010 ,2014 ,2015 ,2016 ,2017 ,2018 ,2019 ,2020 ,2021 ,2022 ,2023],
    "PIB par habitant (%)" : [2.5,1.5,3.3,-0.5,4.0,2.0,1.8,-8.1,7.1,0.5,2.4],
    "PIB par emploi (%)" : [2.6,4.0,4.0,0.9,3.7,2.0,1.3,-3.4,5.7,1.5,None],
    "Taux de chomage (%)" : [9.1 , 9.9,9.7,9.9,10.2,9.5,9.2,11.9,12.3,11.8,13.0],
    "Jeunes NEET (%)" : [None, 28.9,27.9,27.5,29.3,27.5,26.5,26.6,26.0,25.2,25.6],
    "Emploi informel (%)" : [None, 67.6,None,None,None,None,None,None,None,None,None],
    "Banques/100K adultes" : [20.8,24.4,24.9,25.1,25.1,25.2,24.9,24.4,22.4,21.5,20.9],
    "DAB/100k adultes": [19.7, 25.7, 26.5, 27.3, 27.6, 28.2, 29.0, 29.0, 29.4, 29.8, 29.6],
    "PIB du tourisme (%)": [None, 6.7, 6.5, 6.6, 6.8, 6.9, 7.1, None, None, None, None],
   # "Valeur cible 2030": [None, None, None, None, None, None, None, None, None, None, 3.9],
}

df = pd.DataFrame(data)



In [45]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Mise en page du tableau de bord
app.layout = html.Div([
    html.H1("Tableau de bord ODD 8", style={'textAlign': 'center', 'color': '#2c3e50'}),
    
    html.Label("Sélectionnez un indicateur :", style={'marginTop': '10px'}),
    dcc.Dropdown(
        id='indic-dropdown',
        options=[{'label': col, 'value': col} for col in df.columns[1:]],
        value="Taux de chômage (%)",
        clearable=False,
        style={'width': '50%', 'marginBottom': '20px'}
    ),
    
    dcc.Graph(id='line-chart'),
    
    html.Label("Filtrer par année :"),
    dcc.RangeSlider(
        id='year-slider',
        min=df["Année"].min(),
        max=df["Année"].max(),
        step=1,
        marks={i: str(i) for i in range(df["Année"].min(), df["Année"].max() + 1, 2)},
        value=[df["Année"].min(), df["Année"].max()]
    ),
    
    dcc.Graph(id='table-data')
])

# Callback pour mettre à jour le graphique
@app.callback(
    Output('line-chart', 'figure'),
    [Input('indic-dropdown', 'value')]
)
def update_chart(selected_indicator):
    fig = px.line(df, x="Année", y=selected_indicator, markers=True, title=f"Évolution de {selected_indicator}")
    fig.update_traces(line=dict(color='#3498db', width=2), marker=dict(size=8))
    fig.update_layout(template="plotly_white", transition_duration=500)
    return fig

# Callback pour mettre à jour le tableau
@app.callback(
    Output('table-data', 'figure'),
    [Input('year-slider', 'value')]
)
def update_table(selected_years):
    filtered_df = df[(df["Année"] >= selected_years[0]) & (df["Année"] <= selected_years[1])]
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(filtered_df.columns), fill_color='#3498db', align='left', font=dict(color='white')),
        cells=dict(values=[filtered_df[col] for col in filtered_df.columns], fill_color='lavender', align='left')
    )])
    return fig

# Exécution locale
if __name__ == '__main__':
    app.run_server(debug=True)
